In [2]:
from itertools import permutations
from math import exp
from random import shuffle, uniform
import os

class Heuristic:
    def __init__(self, n, time_matrix, dist_matrix):
        self.n = n
        self.time_matrix = time_matrix
        self.dist_matrix = dist_matrix

    @staticmethod
    def load_from_file(path):
        with open(path, "r") as f:
            data = f.read().split("\n")
            data = [i.split() for i in data]
            data = [[int(j) for j in i] for i in data]
            n = data[0][0]
            time_matrix = [[0, 0, 0]] + data[1 : n + 1]
            dist_matrix = data[n + 1 :]
        return Heuristic(n, time_matrix, dist_matrix)

    @staticmethod
    def load_from_input_stream():
        n = int(input())
        data = [input() for _ in range(n * 2 + 1)]
        data = [i.split() for i in data]
        data = [[int(j) for j in i] for i in data]
        time_matrix = [[0, 0, 0]] + data[0:n]
        dist_matrix = data[n:]
        return Heuristic(n, time_matrix, dist_matrix)

    def calculate_total_cost(self, route):
        cur_time, cur_node = 0, 0
        for next_node in route:
            next_time = (
                cur_time
                + self.time_matrix[cur_node][2]
                + self.dist_matrix[cur_node][next_node]
            )
            if next_time > self.time_matrix[next_node][1]:
                return None
            else:
                if next_time < self.time_matrix[next_node][0]:
                    next_time = self.time_matrix[next_node][0]
            cur_time, cur_node = next_time, next_node
        cur_time += self.time_matrix[cur_node][2] + self.dist_matrix[cur_node][0]
        return cur_time

    def generate_routes(self, do_shuffle=True):
        cur_route, cur_cost, visited = [0], 0, [False] * (self.n + 1)

        def Try(k):
            nonlocal cur_route, cur_cost
            loop = list(range(1, self.n + 1))
            if do_shuffle:
                shuffle(loop)
            for i in loop:
                if visited[i]:
                    continue
                added_time = (
                    self.time_matrix[cur_route[-1]][2]
                    + self.dist_matrix[cur_route[-1]][i]
                )
                if cur_cost + added_time > self.time_matrix[i][1]:
                    continue
                if cur_cost + added_time < self.time_matrix[i][0]:
                    added_time = self.time_matrix[i][0] - cur_cost

                cur_route.append(i)
                visited[i] = True
                cur_cost += added_time

                if k == self.n:
                    yield list(cur_route[1:])
                else:
                    yield from Try(k + 1)

                cur_route.pop()
                visited[i] = False
                cur_cost -= added_time

        yield from Try(1)

    def local_search(
        self, init_route, max_iteration=100, debug=False, temperature=None
    ):
        def can_degrade(cur_cost, can_cost, temperature):
            p = exp((cur_cost - can_cost) / temperature)
            r = uniform(0, 1)
            return p > r

        def generate_2opt_change_routes(route):
            route = [0] + route
            n = len(route)
            a = [(i, j) for i in range(n) for j in range(i + 2, n)]
            a.remove((0, n - 1))
            shuffle(a)
            for i, j in a:
                new_route = route[: i + 1] + route[i + 1 : j + 1][::-1] + route[j + 1 :]
                for route in [new_route, new_route[::-1]]:
                    cost = self.calculate_total_cost(route)
                    if cost is not None:
                        route = [x for x in route if x != 0]
                        yield route, cost

        # main code
        cur_route, cur_cost = init_route, self.calculate_total_cost(init_route)
        routes_generator = generate_2opt_change_routes(cur_route)
        iteration = 0
        while True:
            iteration += 1
            if iteration > max_iteration:
                break

            try:
                route, total_cost = next(routes_generator)
            except StopIteration:
                break

            if total_cost < cur_cost:
                cur_route, cur_cost = route, total_cost
                routes_generator = generate_2opt_change_routes(cur_route)
                if debug:
                    print(
                        f"Update route in local search: {cur_route}, cost: {cur_cost}"
                    )
            elif temperature is not None and can_degrade(
                cur_cost, total_cost, temperature
            ):
                cur_route, cur_cost = route, total_cost
                routes_generator = generate_2opt_change_routes(cur_route)
                if debug:
                    print(
                        f"Degrade route in local search: {cur_route}, cost: {cur_cost}"
                    )

        return cur_route, cur_cost

    def iterated_local_search(self, max_iteration=1000, debug=False):
        best_route, best_cost, iteration = None, int(1e9), 0
        for init_route in self.generate_routes():
            iteration += 1
            if iteration > max_iteration:
                break

            route, cost = self.local_search(init_route)
            if cost < best_cost:
                best_route, best_cost = route, cost
                if debug:
                    print(
                        f"Iter: {iteration}, Update best route: {best_route}, cost: {best_cost}"
                    )

        if best_route is not None:
            print(self.n)
            print(f"Best cost: {best_cost}")
            for x in best_route:
                print(x, end=" ")
            print()

    def simulated_annealing(
        self,
        max_search=1000,
        init_temperature=100,
        cooling_factor=0.97,
        min_temperature=0.01,
        debug=False,
    ):
        best_route, best_cost, iteration, temperature = (
            None,
            int(1e9),
            0,
            init_temperature,
        )
        for init_route in self.generate_routes():
            iteration += 1
            if iteration > max_search:
                break

            route, cost = self.local_search(init_route, temperature=temperature)
            if cost < best_cost:
                best_route, best_cost = route, cost
                if debug:
                    print(
                        f"Iter: {iteration}, Update best route: {best_route}, cost: {best_cost}"
                    )

            temperature = max(cooling_factor * temperature, min_temperature)
            # if debug: print(f'Iter: {iteration}, Temperature: {temperature}')

        if best_route is not None:
            print(f'Best cost: {best_cost}')
            print(self.n)
            for x in best_route:
                print(x, end=" ")
            print()


In [4]:
root = '/Users/phamhoang1408/Desktop/Optimization/Plan_Optimize/input/test'
file_paths = ['N5.txt', 'N10.txt']

for file_path in file_paths:
    print(file_path)
    heuristic = Heuristic.load_from_file(os.path.join(root, file_path))
    heuristic.iterated_local_search()
    print('\n' + '-' * 50 + '\n')

N5.txt
5
Best cost: 465
1 5 3 2 4 

--------------------------------------------------

N10.txt
10
Best cost: 779
1 4 2 3 9 10 5 7 6 8 

--------------------------------------------------



In [3]:
root = '/Users/phamhoang1408/Desktop/Optimization/Plan_Optimize/input/test'
file_paths = ['N5.txt', 'N10.txt']

for file_path in file_paths:
    print(file_path)
    heuristic = Heuristic.load_from_file(os.path.join(root, file_path))
    heuristic.simulated_annealing()
    print('\n' + '-' * 50 + '\n')

N5.txt
Best cost: 465
5
1 5 3 2 4 

--------------------------------------------------

N10.txt
Best cost: 779
10
1 4 2 3 9 10 5 7 6 8 

--------------------------------------------------

